# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%matplotlib inline

## Hyperparameter Tunning

In [29]:
num_epochs = 100
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:100]
test_paths = glob('dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Learning Rate Scheduler

In [14]:
import math

In [30]:
def scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.001 * math.exp(0.1 * (10 - epoch))

In [16]:
for epoch in range(10, 20):
    print(0.01 * math.exp(0.1 * (10 - epoch)))

0.001
0.0009048374180359595
0.0008187307530779819
0.0007408182206817179
0.0006703200460356394
0.0006065306597126335
0.0005488116360940264
0.0004965853037914095
0.0004493289641172216
0.00040656965974059914


In [31]:
learning_rate_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

## Training

In [32]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs,
    callbacks=[learning_rate_scheduler]
)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/100
3/3 [==============================] - 0s 59ms/step - loss: 14.6070 - accuracy: 0.0938 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Epoch 2/100
3/3 [==============================] - 0s 52ms/step - loss: 13.9404 - accuracy: 0.1029 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
Epoch 3/100
3/3 [==============================] - 0s 52ms/step - loss: 13.9404 - accuracy: 0.1029 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.
Epoch 4/100
3/3 [==============================] - 0s 51ms/step - loss: 13.9404 - accuracy: 0.1029 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.
Epoch 5/100
3/3 [==============================] - 0s 52ms/step - loss: 13.7552 - accur

3/3 [==============================] - 0s 51ms/step - loss: 13.9404 - accuracy: 0.1029 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00038: LearningRateScheduler reducing learning rate to 6.720551273974975e-05.
Epoch 38/100
3/3 [==============================] - 0s 51ms/step - loss: 14.6070 - accuracy: 0.0938 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00039: LearningRateScheduler reducing learning rate to 6.0810062625217954e-05.
Epoch 39/100
3/3 [==============================] - 0s 50ms/step - loss: 13.7552 - accuracy: 0.1176 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00040: LearningRateScheduler reducing learning rate to 5.502322005640721e-05.
Epoch 40/100
3/3 [==============================] - 0s 52ms/step - loss: 14.1256 - accuracy: 0.0882 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00041: LearningRateScheduler reducing learning rate to 4.9787068367863945e-05.
Epoch 41/100
3/3 [==============================] - 0s 52ms/step - loss: 14.6070 - accuracy: 0.0938

3/3 [==============================] - 0s 51ms/step - loss: 14.1256 - accuracy: 0.0882 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00074: LearningRateScheduler reducing learning rate to 1.8363047770289056e-06.
Epoch 74/100
3/3 [==============================] - 0s 50ms/step - loss: 13.9404 - accuracy: 0.1029 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00075: LearningRateScheduler reducing learning rate to 1.661557273173934e-06.
Epoch 75/100
3/3 [==============================] - 0s 50ms/step - loss: 13.7552 - accuracy: 0.1176 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00076: LearningRateScheduler reducing learning rate to 1.5034391929775723e-06.
Epoch 76/100
3/3 [==============================] - 0s 52ms/step - loss: 14.6070 - accuracy: 0.0938 - val_loss: 14.7749 - val_accuracy: 0.0833

Epoch 00077: LearningRateScheduler reducing learning rate to 1.3603680375478928e-06.
Epoch 77/100
3/3 [==============================] - 0s 53ms/step - loss: 14.1256 - accuracy: 0.088